In [4]:
import numpy as np
import pandas as pd


In [2]:
  from sklearn.datasets import fetch_20newsgroups

  categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

  newsgroups_train = fetch_20newsgroups(subset='train',
                                        remove=('header', 'footers', 'quotes'),
                                        categories=categories)
  
  newsgroups_test = fetch_20newsgroups(subset='test',
                                       remove=('header', 'footers', 'quotes'),
                                        categories=categories)

In [5]:
print(len(newsgroups_train.data), len(newsgroups_test.data))

2034 1353


In [6]:
X_train = newsgroups_train.data
y_train = newsgroups_train.target

X_test = newsgroups_test.data
y_test = newsgroups_test.target

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)
# 5개 미만의 문서에서 나타나는 단어 특성 제외
# 문서의 50%를 초과해 나타나는 단어 제외

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [11]:
for word, count in zip(
    cv.get_feature_names_out()[:100], X_train_cv[0].toarray()[0, :100]
):
  print(word, ':', count, end=',')

00 : 0,000 : 0,01 : 0,02 : 0,04 : 0,05 : 0,10 : 0,100 : 0,11 : 0,12 : 0,128 : 0,129 : 0,13 : 0,130 : 0,14 : 0,15 : 0,16 : 0,17 : 0,18 : 0,19 : 0,1988 : 0,1989 : 0,1990 : 0,1992 : 0,1993 : 0,20 : 0,200 : 0,2000 : 0,202 : 0,21 : 1,22 : 0,23 : 0,24 : 0,25 : 0,256 : 0,26 : 0,27 : 0,28 : 0,29 : 0,2d : 0,30 : 0,300 : 0,31 : 0,32 : 0,33 : 0,34 : 0,35 : 0,36 : 0,37 : 0,38 : 0,39 : 0,3d : 0,3do : 0,40 : 0,408 : 0,41 : 0,42 : 0,43 : 0,44 : 0,45 : 0,47 : 0,48 : 0,49 : 0,50 : 0,500 : 0,51 : 0,52 : 0,55 : 0,56 : 0,60 : 0,65 : 0,70 : 0,75 : 0,80 : 0,800 : 0,90 : 0,91 : 0,92 : 0,93 : 0,95 : 0,_the : 0,ability : 0,able : 1,abortion : 0,about : 1,above : 0,absolute : 0,ac : 1,acad3 : 0,accept : 0,acceptable : 0,accepted : 0,access : 0,according : 0,account : 0,accurate : 0,across : 0,act : 0,action : 0,actions : 0,